In [ ]:
import hist
import time
import json
import dill 
import numpy as np
import awkward as ak
from functools import reduce
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages
import operator

plt.style.use('seaborn-v0_8-white')
plt.rcParams['figure.dpi'] = 200
hep.style.use('CMS')

In [ ]:
def getHist(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

In [ ]:
def draw_hist1d(hist_in, ax, trigger, label='', rebin=1, norm=False):
    hist_in = hist_in[:, trigger, hist.rebin(rebin)]
    counts, _, bins = hist_in.to_numpy()
    if len(counts)>0: # check that there are events to plot
        _counts = counts[0]/(np.sum(counts[0])*np.diff(bins)) if norm else counts[0]
        errs = np.sqrt(counts[0])/(np.sum(counts[0])*np.diff(bins)) if norm else np.sqrt(counts[0])
        _errs = np.where(_counts==0, 0, errs)
        bin_centres = 0.5*(bins[1:] + bins[:-1])
        l = ax.errorbar(x=bin_centres,y=_counts,yerr=_errs,linestyle='')
        color = l[0].get_color()
        ax.errorbar(x=bins[:-1],y=_counts,drawstyle='steps-post',label=label,color=color)
        
    else:        
        l = ax.errorbar(x=[],y=[],yerr=[],drawstyle='steps-post') # plot nothing
        color = l[0].get_color()
        ax.errorbar(x=[],y=[],drawstyle='steps-post',label=label,color=color)

In [ ]:
import matplotlib as mpl
from cycler import cycler

mpl.rcParams['axes.prop_cycle'] = cycler(
    color=[
        '#3f90da', 
        '#ffa90e', 
        '#bd1f01', 
        '#94a4a2', 
        '#832db6', 
        '#a96b59', 
        '#e76300', 
        '#b9ac70', 
        '#717581', 
        '#92dadd'
    ]
)

In [ ]:
dataset = 'Scouting_2024I'
histpath = [dataset, 'hists']

# which triggers to plot (comment out unwanted)
triggers = [
    # 'DST_PFScouting_AXONominal', 
    # 'DST_PFScouting_AXOTight', 
    # 'DST_PFScouting_AXOVTight',
    # 'DST_PFScouting_CICADAMedium', 
    # 'DST_PFScouting_CICADATight', 
    # 'DST_PFScouting_CICADAVTight',
    'DST_PFScouting_ZeroBias'
]
trigger_names = {
    'DST_PFScouting_AXOLoose': 'AXO Loose', 
    'DST_PFScouting_AXONominal': 'AXO Nominal', 
    'DST_PFScouting_AXOTight': 'AXO Tight', 
    'DST_PFScouting_AXOVLoose': 'AXO VLoose',
    'DST_PFScouting_AXOVTight': 'AXO VTight',
    'DST_PFScouting_CICADAMedium': 'CICADA Medium', 
    'DST_PFScouting_CICADATight': 'CICADA Tight', 
    'DST_PFScouting_CICADAVTight': 'CICADA VTight', 
    'DST_PFScouting_DoubleMuon': 'Double Muon',
    'DST_PFScouting_JetHT': 'Jet HT',
    'DST_PFScouting_ZeroBias': 'Zero Bias'
}

In [ ]:
with open(f'hist_result_{dataset}_test.pkl', 'rb') as file:
    hist_result = dill.load(file)

In [ ]:
hist_result['Scouting_2024I']['hists'].keys()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 10))
hist_name =  'l1ht'
norm = False#

for trigger in triggers:
    print(trigger)
    hist_current = getHist(hist_result, histpath+[hist_name])
    draw_hist1d(hist_current, ax[0], trigger, label=trigger_names[trigger], rebin=1, norm=norm)

hep.cms.label('', ax=ax[0], data=True, lumi=None, year=dataset, com=13.6)
ax[0].set_xlim(0,250)
ax[0].set_ylabel(f'Events{" [A.U.]" if norm else ""}', loc='top')
ax[0].set_xlabel('L1 HT', loc='right')
# ax[0].set_yscale('log')
ax[0].legend();
ax[1].set_visible(False)